In [ ]:
import numpy as np
import cv2
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance
import textwrap
import os
import json
from tqdm import tqdm
import requests
import zipfile
import io
import random
from functools import lru_cache

class TextAnimationVideoGenerator:
    def __init__(self,
                 width=1080,  # Standard shorts/vertical format width
                 height=1920, # Standard shorts/vertical format height
                 fps=20,
                 font_size=100,
                 text_color=(255, 255, 255),
                 bg_color=(0, 0, 0),
                 chars_per_frame=0.5,
                 scroll_speed=1,
                 margin_top=100,
                 margin_bottom=100,
                 margin_left=50,
                 margin_right=50,
                 text_effects=True):
        self.width = width
        self.height = height
        self.fps = fps
        self.text_color = text_color
        self.bg_color = bg_color
        self.chars_per_frame = chars_per_frame
        self.scroll_speed = scroll_speed
        self.margin_top = margin_top
        self.margin_bottom = margin_bottom
        self.margin_left = margin_left
        self.margin_right = margin_right
        self.text_effects = text_effects

        # Dynamic elements
        self.frame_count = 0

        # Visual style settings
        self.glow_amount = 3
        self.shadow_offset = 3
        self.shadow_color = (0, 0, 0, 180)  # Semi-transparent black

        # Font size - this is now a base size
        self.font_size = font_size
        self.font = None
        self.title_font = None

        # Set up usable dimensions - these won't change
        self.usable_width = self.width - (self.margin_left + self.margin_right)
        self.usable_height = self.height - self.margin_top - self.margin_bottom

        # Fixed width in pixels rather than characters
        self.target_line_width = self.usable_width * 0.9  # Use 90% of usable width

        # Background textures for page effect
        self.page_textures = []
        self.load_or_create_page_textures()

        # Precompute backgrounds to improve performance
        self.background_cache = {}

        # Store book number for debugging
        self.current_book_num = 0

    def load_or_create_page_textures(self):
        """Load or create page textures for the page turning effect"""
        for i in range(3):
            texture = Image.new('RGB', (self.width, self.height), (245, 240, 230))
            draw = ImageDraw.Draw(texture)

            # Add subtle noise to simulate paper texture
            for _ in range(5000):
                x = random.randint(0, self.width - 1)
                y = random.randint(0, self.height - 1)
                size = random.randint(1, 3)
                opacity = random.randint(5, 15)
                draw.rectangle([(x, y), (x + size, y + size)],
                               fill=(245 - opacity, 240 - opacity, 230 - opacity))

            # Add subtle paper grain lines
            for _ in range(20):
                y = random.randint(0, self.height - 1)
                opacity = random.randint(5, 10)
                draw.line([(0, y), (self.width, y)],
                          fill=(245 - opacity, 240 - opacity, 230 - opacity), width=1)

            self.page_textures.append(texture)

    def load_book_from_json(self, json_path):
        """Load book information from a JSON file with improved error handling"""
        try:
            # Extract book number from path for debugging
            self.current_book_num = int(os.path.basename(json_path).split('_')[1].split('.')[0])
            print(f"Loading book number {self.current_book_num}")

            with open(json_path, 'r', encoding='utf-8') as file:
                book_data = json.load(file)

            # Use .get() to provide defaults if keys are missing
            title = book_data.get('title', 'Untitled')
            author = book_data.get('author', 'Unknown Author')
            synopsis = book_data.get('synopsis', '')
            book_text = f"{title}\nby {author}\n\n{synopsis}"

            # Load font information
            font_url = book_data.get('font', {}).get('url', None)
            font_path = self.download_font(font_url) if font_url else None

            return {
                'text': book_text,
                'font_path': font_path,
                'book_data': book_data
            }
        except Exception as e:
            print(f"Error loading book data from {json_path}: {e}")
            return None

    def load_font(self, font_path, font_size):
        """Load font or download it if specified by URL"""
        try:
            if font_path and font_path.startswith(('http://', 'https://')):
                font_path = self.download_font(font_path)

            if font_path and os.path.exists(font_path):
                # Create fonts at the given size
                self.font = ImageFont.truetype(font_path, font_size)
                self.title_font = ImageFont.truetype(font_path, int(font_size * 1.5))
                print(f"Book {self.current_book_num}: Successfully loaded font: {font_path} at size {font_size}px")
            else:
                raise Exception(f"Font path not found: {font_path}")
        except Exception as e:
            print(f"Font loading error: {e}. Using default font.")
            try:
                system_fonts = ['Arial.ttf', 'Verdana.ttf', 'DejaVuSans.ttf', '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf']
                for font in system_fonts:
                    try:
                        self.font = ImageFont.truetype(font, font_size)
                        self.title_font = ImageFont.truetype(font, int(font_size * 1.5))
                        print(f"Using system font: {font}")
                        return
                    except:
                        continue
            except:
                pass
            self.font = ImageFont.load_default()
            self.title_font = ImageFont.load_default()
            if hasattr(self.font, "size"):
                self.font = self.font.font_variant(size=font_size)
                self.title_font = self.font.font_variant(size=int(font_size * 1.5))

    def download_font(self, url):
        """Download a font from URL"""
        if not url:
            return None
        try:
            print(f"Downloading font from {url}...")
            if "fonts.google.com/specimen" in url:
                font_family = url.split('/')[-1].split('?')[0].replace('+', ' ')
                api_url = f"https://fonts.googleapis.com/css?family={font_family.replace(' ', '+')}"
                css_response = requests.get(api_url)
                css_response.raise_for_status()
                import re
                font_url_match = re.search(r'url\((.*?\.ttf)\)', css_response.text)
                if font_url_match:
                    url = font_url_match.group(1)
                else:
                    print(f"Couldn't extract font URL from Google Fonts CSS")
                    return None
            response = requests.get(url)
            response.raise_for_status()
            if url.endswith('.zip') or "fonts.google.com/download" in url:
                z = zipfile.ZipFile(io.BytesIO(response.content))
                font_files = [f for f in z.namelist() if f.endswith(('.ttf', '.otf'))]
                if not font_files:
                    raise Exception("No font files found in zip")
                font_path = "downloaded_font.ttf"
                with open(font_path, 'wb') as f:
                    f.write(z.read(font_files[0]))
                print(f"Extracted {font_files[0]} from zip as {font_path}")
            else:
                font_path = os.path.basename(url) if "." in os.path.basename(url) else "downloaded_font.ttf"
                with open(font_path, 'wb') as f:
                    f.write(response.content)
            print(f"Font downloaded and saved as {font_path}")
            return font_path
        except Exception as e:
            print(f"Error downloading font: {e}")
            return None

    def calculate_appropriate_font_size(self, font_path, sample_text="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"):
        """Calculate an appropriate font size to fit approximately 28 characters per line"""
        try:
            # Start with a test font size
            test_size = self.font_size
            test_font = ImageFont.truetype(font_path, test_size)

            # Calculate the width of our sample text
            if hasattr(test_font, "getlength"):
                text_width = test_font.getlength(sample_text)
            elif hasattr(test_font, "getsize"):
                text_width = test_font.getsize(sample_text)[0]
            else:
                # Fallback for older PIL versions
                img = Image.new('RGB', (1, 1))
                draw = ImageDraw.Draw(img)
                text_width = draw.textlength(sample_text, font=test_font)

            # Calculate width per character
            char_width = text_width / len(sample_text)

            # Calculate desired character width to fit ~28 characters in target_line_width
            target_char_width = self.target_line_width / 28

            # Calculate adjusted font size
            adjusted_size = int(test_size * (target_char_width / char_width))

            print(f"Book {self.current_book_num}: Adjusted font size from {test_size} to {adjusted_size} for consistent width")

            # Add bounds to prevent extreme sizes
            adjusted_size = max(40, min(80, adjusted_size))

            return adjusted_size
        except Exception as e:
            print(f"Error calculating font size: {e}. Using default size {self.font_size}")
            return self.font_size

    def setup_font_and_metrics(self, font_path=None):
        """Set up font and calculate text metrics with fixed width"""
        if font_path and os.path.exists(font_path):
            # Calculate font size based on desired characters per line
            adjusted_font_size = self.calculate_appropriate_font_size(font_path)
            self.load_font(font_path, adjusted_font_size)
        else:
            self.load_font(font_path, self.font_size)

        # Calculate line height based on font
        test_text = "AygjpqQ|"
        if hasattr(self.font, "getbbox"):
            bbox = self.font.getbbox(test_text)
            self.text_height = bbox[3] - bbox[1]
        else:
            bbox = self.font.getmask(test_text).getbbox()
            self.text_height = bbox[3]

        self.line_height = int(self.text_height * 1.3)

        # Calculate chars_per_line based on font metrics
        test_text = "m" * 30  # Using 'm' as it's typically a wide character
        if hasattr(self.font, "getlength"):
            avg_char_width = self.font.getlength(test_text) / len(test_text)
        elif hasattr(self.font, "getsize"):
            avg_char_width = self.font.getsize(test_text)[0] / len(test_text)
        else:
            # Fallback method for older PIL versions
            img = Image.new('RGB', (1, 1))
            draw = ImageDraw.Draw(img)
            avg_char_width = draw.textlength(test_text, font=self.font) / len(test_text)

        # Calculate chars that fit in the target line width
        self.chars_per_line = max(28, min(40, int(self.target_line_width / avg_char_width)))

        # Calculate lines that fit on screen
        self.lines_per_screen = max(5, self.usable_height // self.line_height)

        print(f"Book {self.current_book_num}: Font metrics: text height={self.text_height}px, line height={self.line_height}px")
        print(f"Book {self.current_book_num}: Screen capacity: {self.chars_per_line} chars per line, {self.lines_per_screen} lines per screen")
        print(f"Book {self.current_book_num}: Average character width: {avg_char_width:.2f}px")

    def get_text_dimensions(self, text, font):
        """Get the width and height of a text string with the given font"""
        if hasattr(font, "getbbox"):
            bbox = font.getbbox(text)
            return bbox[2] - bbox[0], bbox[3] - bbox[1]
        elif hasattr(font, "getsize"):
            return font.getsize(text)
        else:
            # Fallback method for older PIL versions
            img = Image.new('RGB', (1, 1))
            draw = ImageDraw.Draw(img)
            return draw.textlength(text, font=font), self.text_height

    def process_text(self, text):
        """Process text into lines with appropriate wrapping and styling"""
        text = text.replace("\\n", "\n")
        paragraphs = text.split("\n")
        wrapped_lines = []
        is_title = True
        is_author = False

        for paragraph in paragraphs:
            if not paragraph.strip():
                wrapped_lines.append({"text": "", "style": "normal"})
                continue

            style = "normal"
            if is_title and paragraph.strip():
                style = "title"
                is_title = False
                is_author = True
            elif is_author and paragraph.strip():
                style = "author"
                is_author = False

            # Use different wrapping widths based on style
            wrap_width = self.chars_per_line
            if style == "title":
                # Title tends to use larger font, so fewer characters per line
                wrap_width = max(10, int(self.chars_per_line * 0.7))

            wrapped = textwrap.wrap(paragraph, width=wrap_width)
            if wrapped:
                for line in wrapped:
                    wrapped_lines.append({"text": line, "style": style})
                wrapped_lines.append({"text": "", "style": "normal"})

        # Remove trailing empty line if exists
        if wrapped_lines and wrapped_lines[-1]["text"] == "":
            wrapped_lines.pop()

        return wrapped_lines

    def create_background(self, variant_key=0):
        """Create a background with paper texture"""
        texture_idx = variant_key % len(self.page_textures)
        return self.page_textures[texture_idx].copy()

    def draw_text_with_effects(self, img, text, position, font, color, style="normal", center_horizontally=True):
        """Draw text with enhanced visual effects and optional horizontal centering"""
        x, y = position
        if center_horizontally:
            text_width, _ = self.get_text_dimensions(text, font)
            x = (self.width - text_width) // 2

        if not self.text_effects:
            draw = ImageDraw.Draw(img)
            draw.text((x, y), text, font=font, fill=color)
            return

        if style == "title":
            text_color = (20, 20, 80, 255)
            shadow_color = (150, 150, 150, 100)
            shadow_offset = 2
        elif style == "author":
            text_color = (70, 70, 120, 255)
            shadow_color = (180, 180, 180, 80)
            shadow_offset = 1
        else:
            text_color = (40, 40, 40, 255)
            shadow_color = None
            shadow_offset = 0

        # Get text dimensions for accurate image sizing
        text_width, text_height = self.get_text_dimensions(text, font)

        # Create an image with sufficient padding
        text_img = Image.new('RGBA', (int(text_width + 50), int(self.line_height * 2)), (0, 0, 0, 0))
        text_draw = ImageDraw.Draw(text_img)

        # Draw shadow if needed
        if shadow_color and shadow_offset > 0:
            text_draw.text((shadow_offset, shadow_offset), text, font=font, fill=shadow_color)

        # Draw main text
        text_draw.text((0, 0), text, font=font, fill=text_color)

        # Paste onto main image
        img.paste(text_img, (int(x), int(y)), text_img)

    def create_frame(self, visible_text, current_typing_line="", current_typing_pos=0, page_turn_progress=0):
        """Create a single frame with the visible text centered on the page"""
        self.frame_count += 1
        background_variation = (self.frame_count // 150) % len(self.page_textures)
        img = self.create_background(background_variation)

        if page_turn_progress > 0:
            return self.create_page_turn_frame(img, visible_text, page_turn_progress)

        # Calculate total height of all visible text
        total_height = 0
        for line in visible_text:
            if not line["text"].strip():
                total_height += self.line_height
                continue

            style = line["style"]
            total_height += self.line_height * (1.5 if style == "title" else 1.0)

        # Add height for currently typing line if present
        if current_typing_line:
            total_height += self.line_height

        # Calculate starting y-position to center vertically
        y_position = max(self.margin_top, (self.height - total_height) // 2)

        for line in visible_text:
            text = line["text"]
            style = line["style"]
            if not text.strip():
                y_position += self.line_height
                continue

            current_font = self.title_font if style == "title" and hasattr(self, 'title_font') else self.font
            self.draw_text_with_effects(img, text, (0, y_position), current_font, self.text_color, style, center_horizontally=True)

            # Add extra spacing after title
            y_position += self.line_height * (1.5 if style == "title" else 1.0)

        if current_typing_line:
            if isinstance(current_typing_line, dict):
                text = current_typing_line["text"][:int(current_typing_pos)]
                style = current_typing_line["style"]
            else:
                text = current_typing_line[:int(current_typing_pos)]
                style = "normal"

            current_font = self.title_font if style == "title" and hasattr(self, 'title_font') else self.font

            # Add cursor blink effect
            if self.frame_count % self.fps < self.fps / 2:
                text += "|"

            self.draw_text_with_effects(img, text, (0, y_position), current_font, self.text_color, style, center_horizontally=True)

        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    def create_page_turn_frame(self, base_img, visible_text, progress):
        """Create a frame showing a page turning effect with centered text"""
        current_page = base_img.copy()

        # Calculate total height of all visible text for vertical centering
        total_height = 0
        for line in visible_text:
            if not line["text"].strip():
                total_height += self.line_height
                continue

            style = line["style"]
            total_height += self.line_height * (1.5 if style == "title" else 1.0)

        # Calculate starting y-position to center vertically
        y_position = max(self.margin_top, (self.height - total_height) // 2)

        for line in visible_text:
            text = line["text"]
            style = line["style"]
            if not text.strip():
                y_position += self.line_height
                continue

            current_font = self.title_font if style == "title" and hasattr(self, 'title_font') else self.font
            self.draw_text_with_effects(current_page, text, (0, y_position), current_font, self.text_color, style, center_horizontally=True)
            y_position += self.line_height * (1.5 if style == "title" else 1.0)

        next_page = self.create_background((self.frame_count // 150 + 1) % len(self.page_textures))
        turn_width = int(self.width * progress)
        shadow_width = 30
        result = current_page.copy()
        turn_mask = Image.new('L', (self.width, self.height), 0)
        turn_mask_draw = ImageDraw.Draw(turn_mask)
        turn_mask_draw.rectangle([(self.width - turn_width, 0), (self.width, self.height)], fill=255)
        shadow_img = Image.new('RGBA', (self.width, self.height), (0, 0, 0, 0))
        shadow_draw = ImageDraw.Draw(shadow_img)
        for i in range(shadow_width):
            alpha = int(80 * (1 - i/shadow_width))
            shadow_draw.line([(self.width - turn_width - i, 0), (self.width - turn_width - i, self.height)], fill=(0, 0, 0, alpha))
        result.paste(next_page, (0, 0), turn_mask)
        result = Image.alpha_composite(result.convert('RGBA'), shadow_img).convert('RGB')
        curve_img = Image.new('RGBA', (self.width, self.height), (0, 0, 0, 0))
        curve_draw = ImageDraw.Draw(curve_img)
        curve_width = min(20, int(turn_width/3))
        for i in range(curve_width):
            alpha = int(40 * (1 - i/curve_width))
            curve_draw.line([(self.width - turn_width + i, 0), (self.width - turn_width + i, self.height)], fill=(255, 255, 255, alpha))
        result = Image.alpha_composite(result.convert('RGBA'), curve_img).convert('RGB')
        return cv2.cvtColor(np.array(result), cv2.COLOR_RGB2BGR)

    def generate_video(self, text, output_filename="book_promo.mp4", duration=None):
        """Generate a video with text animation and page turning effect"""
        lines = self.process_text(text)
        output_dir = os.path.dirname(output_filename)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
        try:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))
            if not video.isOpened():
                print("Trying H264 codec...")
                fourcc = cv2.VideoWriter_fourcc(*'H264')
                video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))
            if not video.isOpened():
                print("Falling back to AVI format...")
                output_filename = output_filename.replace('.mp4', '.avi')
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                video = cv2.VideoWriter(output_filename, fourcc, self.fps, (self.width, self.height))
            if not video.isOpened():
                raise Exception("Failed to create video writer with any codec")
        except Exception as e:
            print(f"Error initializing video writer: {e}")
            raise
        if duration:
            total_frames = int(duration * self.fps)
            total_chars = sum(len(line["text"]) for line in lines)
            self.chars_per_frame = max(1, total_chars / total_frames * 0.5)
            print(f"Book {self.current_book_num}: Adjusted to {self.chars_per_frame} chars per frame to fit {duration}s duration")
        total_frames_estimate = int(sum(len(line["text"]) for line in lines) / self.chars_per_frame * 1.1)
        with tqdm(total=total_frames_estimate, desc=f"Generating video frames for Book {self.current_book_num}") as pbar:
            current_line_index = 0
            char_index = 0
            visible_lines = []
            while current_line_index < len(lines):
                current_line = lines[current_line_index]
                need_page_turn = len(visible_lines) >= self.lines_per_screen - 1 and char_index >= len(current_line["text"])
                if char_index >= len(current_line["text"]):
                    visible_lines.append(current_line)
                    current_line_index += 1
                    char_index = 0
                if need_page_turn:
                    pause_frames = int(self.fps * 0.5)
                    for _ in range(pause_frames):
                        frame = self.create_frame(visible_lines)
                        if frame is not None:
                            video.write(frame)
                        pbar.update(0.1)
                    page_turn_frames = int(self.fps * 1.0)
                    for i in range(page_turn_frames):
                        progress = i / page_turn_frames
                        frame = self.create_frame(visible_lines, page_turn_progress=progress)
                        if frame is not None:
                            video.write(frame)
                        pbar.update(0.1)
                    visible_lines = []
                if not need_page_turn:
                    frame = self.create_frame(visible_lines, current_line, char_index)
                    if frame is not None:
                        video.write(frame)
                char_index += self.chars_per_frame
                pbar.update(1)
            pause_frames = int(self.fps * 2)
            for _ in range(pause_frames):
                frame = self.create_frame(visible_lines)
                if frame is not None:
                    video.write(frame)
                pbar.update(1)
        video.release()
        print(f"Video saved as {output_filename}")
        return output_filename

def generate_book_videos(start_book=1, end_book=1, storage_path="Storage/temp_texts", output_path="Storage/temp_videos"):
    """Generate videos for multiple books with a fresh generator for each book"""
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    if not os.path.exists(storage_path):
        os.makedirs(storage_path)

    for book_num in range(start_book, end_book + 1):
        # Create a new generator for each book to prevent settings from carrying over
        generator = TextAnimationVideoGenerator(
            width=1080,
            height=1920,
            fps=20,
            font_size=60,  # This is now a base size that will be adjusted
            chars_per_frame=0.5,
            bg_color=(245, 240, 230),
            text_color=(40, 40, 40),
            margin_top=150,
            margin_bottom=200,
            margin_left=70,
            margin_right=70,
            text_effects=True
        )

        book_json_path = os.path.join(storage_path, f"book_{book_num}.json")
        if not os.path.exists(book_json_path):
            print(f"Book file {book_json_path} not found. Skipping.")
            continue

        book_data = generator.load_book_from_json(book_json_path)
        if not book_data:
            print(f"Failed to load book data for book {book_num}. Skipping.")
            continue

        generator.setup_font_and_metrics(book_data['font_path'])
        output_filename = os.path.join(output_path, f"book_{book_num}.mp4")

        try:
            generator.generate_video(book_data['text'], output_filename=output_filename, duration=38)
            print(f"Completed video for Book {book_num}")
        except Exception as e:
            print(f"Error generating video for Book {book_num}: {e}")

def main():
    start_book = 1
    end_book = 5
    generate_book_videos(start_book=start_book, end_book=end_book, storage_path="Storage/temp_texts", output_path="Storage/temp_videos")

if __name__ == "__main__":
    main()
